
# **Gen** ret<strong><font color="red">AI</font></strong>l

Customer search:
* I search flower for **weading.**
* I search flower for **mother day.**
* I search flower for **valentine day.**

Company provide:
* I found the same product <strong><font color="red">contextualized</font></strong> to my search

In [1]:
%pip install --user --upgrade gradio

Note: you may need to restart the kernel to use updated packages.


# Init project

In [3]:
# init the google cloud project id

def init_gcloud_(project_id = "ucs-fishfood-8"):
    """
    Initialize the google cloud project id.
    """
    # [START gae_python37_app_init_gcloud]
    !gcloud config set project $project_id

def init_project_genappbuilder():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "ucs-fishfood-8")


def init_project_images():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "cloud-llm-preview4")


def init_project_workstations():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "cloud-workstations-demo-2")

def init_project_ml_api():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "google.com:ml-baguette-demos")

    

# Gen App builder code to search product on website

In [32]:
import requests
import json
import time

#init_project_genappbuilder()

def genAppBuilder(query , ACCESS_TOKEN = None):
    

    if ACCESS_TOKEN == None:
      gcloud_token = !gcloud auth print-access-token
      gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()

      ACCESS_TOKEN = gcloud_token[0]
    
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {"query": query, "page_size": "5", "offset": 0 }
    
    #print(data)
    url = "https://discoveryengine.googleapis.com/v1alpha/projects/210552312048/locations/global/collections/default_collection/dataStores/interflora_website/servingConfigs/default_search:search" 
    url="https://discoveryengine.googleapis.com/v1alpha/projects/660199673046/locations/global/collections/default_collection/dataStores/bienici_1686652101046/servingConfigs/default_search:search"
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    #print(response)
    json_response = json.loads(response.text)
    print(json_response)
    
    title, link, snippet, description = "", "", "", ""
    try:
      predictions = json_response['results']
            
      link = prediction["document"]["derivedStructData"]["link"]
      title = prediction["document"]["derivedStructData"]["title"]
      snippet = prediction["document"]["derivedStructData"]["snippets"][0]['snippet']
      description = prediction["document"]["derivedStructData"]['pagemap']['metatags'][0]["twitter:description"]
      image = predictions[0]["document"]["derivedStructData"]['pagemap']['cse_image'][0]["src"]



    except:
      print("An error occured calling the API.")
      print("1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...")
      print("2. Try a different prompt to see if that was the problem.\n")
      print(response.text)
      # print(dir(response))

    return title, link, snippet, description, image
    

TOKEN= GAPP_BUILDER_API_KEY


title, link, snippet, description, image = genAppBuilder("fleur de mariage", TOKEN)
print("Title: ", title)
print("Link: ", link)
print("Description: ", description)
print("Snippet: ", snippet)
print("Image: ", image)







{'results': [{'id': '0', 'document': {'name': 'projects/660199673046/locations/global/collections/default_collection/dataStores/bienici_1686652101046/branches/0/documents/0', 'id': '0', 'derivedStructData': {'snippets': [{'snippet': 'Mar 16, 2023 ... Véhicule pour livraisons. Activités : Fleurs coupées, compositions florales, cadeaux, mariage, deuils et autres évènements, plantes.', 'htmlSnippet': 'Mar 16, 2023 <b>...</b> Véhicule <b>pour</b> livraisons. Activités : <b>Fleurs</b> coupées, compositions florales, cadeaux, <b>mariage</b>, deuils et autres évènements, plantes.'}], 'link': 'https://www.bienici.com/annonce/vente/guerande/fonds-de-commerce/ag311656-379069684', 'displayLink': 'www.bienici.com', 'formattedUrl': 'https://www.bienici.com/annonce/vente/guerande/.../ag311656-379069684', 'htmlTitle': 'Achat fonds <b>de</b> commerce, Guérande - 45 000 €', 'pagemap': {'cse_image': [{'src': 'https://file.bienici.com/photo/ag311656-379069684_photos.ubiflow.net_311656_379069684_photos_1.

UnboundLocalError: local variable 'image' referenced before assignment

In [27]:
import os

GAPP_BUILDER_API_KEY = "ya29.a0AWY7CkkaTZN65Vb-CuuC5olSU957-bZNuDBJVVbjV4k0jJI7phzHIC6HVPGCfFBfsHdTyN_lsqsazVI3ayjf5XcDarMi2_hNpscnOxjZT2GVgVZSZYincSRm1CnhAK0TO8X2w3fLB_SzwDVZfA9r0XKT_0CmBrCzr3MtLlk87yS48EwrRkcQQi5_HnB98faJIpFaEIO0dK0Ty9XGxXwsbR21C1pIqS-bbGmEMi2RS0f1JNC6PaOe8NxagbvgH9uLRpMxUitrwhxl49px2CuBQyq-TwnZSJ5Fvv9-1iM-rtWANqSNv4bj19H8UOb6_nMjvSmvCUxERUKNCNKDnh9b1RW0rDdrJKk0f4vaQFxbSsRnugPv-F2YkPp-dEMHdEbN9rVfRakB4oWNuF-4w8J5x9IJbhcaCgYKAbESAQ8SFQG1tDrp5Wx6GqDfC54KY1sp8JNXCg0418"

os.environ["GAPP_BUILDER_API_KEY"]= GAPP_BUILDER_API_KEY

In [28]:
"""Tool for the Google search API."""

from typing import Optional, Type
from pydantic import BaseModel, Field

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
#from langchain.utilities.google_appbuilder_api import GoogleAppBuilderAPIWrapper


class GoogleAppBuilderSchema(BaseModel):
    query: str = Field(..., description="Query for google app builder")


class GoogleAppBuilderTool(BaseTool):
    """Tool that adds the capability to query the Google appbuilders API."""

    name = "google_appbuilder"
    description = (
        "A wrapper around Google appbuilders. "
        "Useful for when you need to validate or "
        "discover addressed from ambiguous text. "
        "Input should be a search query."
    )
    api_wrapper: GoogleAppBuilderAPIWrapper = GoogleAppBuilderAPIWrapper() #Field(default_factory=GoogleAppBuilderAPIWrapper)
    args_schema: Type[BaseModel] = GoogleAppBuilderSchema

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return self.api_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("GoogleAppBuilderRun does not support async")


In [5]:

import logging
from typing import Any, Dict, Optional

from pydantic import BaseModel, Extra, root_validator

from langchain.utils import get_from_dict_or_env


 
class GoogleAppBuilderAPIWrapper(BaseModel):
    """Wrapper around Google AppBuilder API.

    To use, you should have 
     **an API key for the google AppBuilder platform**,
     and the enviroment variable ''GAPP_BUILDER_API_KEY''
     set with your API key , or pass 'gappbuilders_api_key'
     as a named parameter to the constructor.

    By default, this will return the all the results on the input query.
     You can use the top_k_results argument to limit the number of results.

    Example:
        .. code-block:: python


            from langchain import GoogleAppBuilderAPIWrapper
            gappbuilderapi = GoogleAppBuilderAPIWrapper()
    """

    gappbuilders_api_key: Optional[str] = None
    google_map_client: Any  #: :meta private:
    top_k_results: Optional[int] = None
    url="https://discoveryengine.googleapis.com/v1alpha/projects/660199673046/locations/global/collections/default_collection/dataStores/bienici_1686652101046/servingConfigs/default_search:search"

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:


        """Validate that api key is in your environment variable."""
        gappbuilders_api_key = get_from_dict_or_env(
            values, "gappbuilders_api_key", "GAPP_BUILDER_API_KEY"
        )
        values["gappbuilders_api_key"] = gappbuilders_api_key
        
        return values

    def run(self, query: str) -> str:
        """Run appbuilders search and get k number of appbuilders that exists that match."""

    
        headers = {
            'Authorization': f'Bearer {self.gappbuilders_api_key}',
            'Content-Type': 'application/json; charset=UTF-8'
        }
        print(headers)
        page_size = self.top_k_results
        print(f"page_size : {page_size}")
        page_size = 1
        data = {"query": query, "page_size": f"{page_size}", "offset": 0 }
        
        #print(data)
        response = requests.post(self.url, data=json.dumps(data), headers=headers)
        
        #print(response)
        search_results = json.loads(response.text)
        #print(search_results)
        search_results = search_results.get("results", {})

        num_to_return = len(search_results)

        print(search_results)
        appbuilders = []

        if num_to_return == 0:
            return "Google appbuilders did not find any appbuilders that match the description"

        num_to_return = 5 
#        num_to_return = (            num_to_return
#            if self.top_k_results is None
#            else min(num_to_return, self.top_k_results)        )

        for i in range(num_to_return):

            result = search_results[id==i]
            details = self.fetch_appbuilder_details(result)

            if details is not None:
                appbuilders.append(details)

        return "\n".join([f"{i+1}. {item}" for i, item in enumerate(appbuilders)])

    def fetch_appbuilder_details(self, prediction: str) -> Optional[str]:
        try:
           
            derivedStructData = prediction.get("document", {}).get("derivedStructData", {})

            formatted_details = self.format_appbuilder_details(derivedStructData)
            return formatted_details
        except Exception as e:
            logging.error(f"An Error occurred while fetching appbuilder details: {e}")
            return None
        


    def format_appbuilder_details(self, appbuilder_details: Dict[str, Any]) -> Optional[str]:
        try:
            title = appbuilder_details.get("title", "Unkown")

            formatted_details = title
            formatted_details = formatted_details +"\n"+ self.format_array(appbuilder_details)            
                
            return formatted_details
            pagemap =  appbuilder_details.get("pagemap", {})
            
            metatags = self.fetch_strucdata(pagemap.get("metatags", {}))
            snippets = self.fetch_strucdata(pagemap.get("snippet", {}))
            link = self.fetch_strucdata(pagemap.get("link", {}))
                                        
            image = self.fetch_strucdata(pagemap.get("image", {}))                
            
            formatted_details = (
                f"{title}\nDescription: {metatags}\n"
                f"snippets: {snippets}\n"
                f"image: {image}\n"                
                                f"source: {link}\n\n"
            )
            return formatted_details
        except Exception as e:
            logging.error(f"An error occurred while formatting appbuilder details: {e}")
            return None

    def format_array(self, appbuilder_details):
        format_array = ""
        keys = appbuilder_details.keys()
        for key in keys:
            value = appbuilder_details[key]
            if value is str:
                print(f"{key} : {value}\n")
                formatted_details = (formatted_details + 
                                         f"{key} : {value}")
            else:
                formatted_details =  format_array(value)
        return formatted_details

    def fetch_strucdata(self, metatags):
        description = ""
        for metatag in  metatags:                
            description += metatag.get("twitter:description", "Unkown") + "\n"
        
        return description


#api = GoogleAppBuilderAPIWrapper()
#api.run("maison à vendre plérin 2 voiture")


def format_array(appbuilder_details):
    format_array = ""
    keys = appbuilder_details.keys()
    for key in keys:
        value = appbuilder_details[key]
        if value is str:
            print(f"{key} : {value}\n")
            formatted_details = (formatted_details + 
                                        f"{key} : {value}")
        else:
            formatted_details =  format_array(value)
    return formatted_details

def format_appbuilder_details( appbuilder_details: Dict[str, Any]) -> Optional[str]:
    title = appbuilder_details.get("title", "Unkown")

    formatted_details = title
    formatted_details = formatted_details +"\n"+ format_array(appbuilder_details)            
        
    return formatted_details



def fetch_appbuilder_details( prediction: str) -> Optional[str]:
    try:
        
        derivedStructData = prediction.get("document", {}).get("derivedStructData", {})

        formatted_details = format_appbuilder_details(derivedStructData)
        return formatted_details
    except Exception as e:
        logging.error(f"An Error occurred while fetching appbuilder details: {e}")
        return None

search_results = [{'id': '0', 'document': {'name': 'projects/660199673046/locations/global/collections/default_collection/dataStores/bienici_1686652101046/branches/0/documents/0', 'id': '0', 'derivedStructData': {'snippets': [{'snippet': "A VENDRE, PLERIN, 2mn de la plage en voiture, maison contemporaine de 1993 ... en 1/2 niveau: coin bureau, une chambre avec dressing, wc, salle d'eau;\xa0...", 'htmlSnippet': 'A <b>VENDRE</b>, <b>PLERIN</b>, 2mn de la plage en <b>voiture</b>, <b>maison</b> contemporaine de 1993 ... en 1/<b>2</b> niveau: coin bureau, une chambre avec dressing, wc, salle d&#39;eau;&nbsp;...'}], 'link': 'https://www.bienici.com/annonce/vente/plerin/maison/9pieces/immo-facile-50380739', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQ9KUzSFkEH-Ep4UaCRX26sJ1NlDENc82NJR1MQ1N2PqlTscuIoLE1Ynpqr', 'height': '194', 'width': '259'}], 'cse_image': [{'src': 'https://file.bienici.com/photo/immo-facile-50380739_media.immo-facile.com_office_sbi_catalog_images_pr_p_5_0_3_8_0_7_3_9_50380739a.jpg_DATEMAJ_19_07_2022-20_04_47'}], 'metatags': [{'apple-mobile-web-app-capable': 'yes', 'og:title': 'Achat maison 9\xa0pièces 235\xa0m², Plérin - 760\xa0900\xa0€', 'apple-mobile-web-app-status-bar-style': 'black', 'title': 'Achat maison 9\xa0pièces 235\xa0m², Plérin - 760\xa0900\xa0€', 'fb:app_id': '1650758831830906', 'sentry-dsn': 'https://8ce0df5c8a4a49c7a47c360691be1f76@sentry.io/154017', 'twitter:title': 'Achat maison 9\xa0pièces 235\xa0m², Plérin - 760\xa0900\xa0€', 'apple-itunes-app': 'app-id=1181371766', 'theme-color': '#ffffff', 'twitter:image': 'https://file.bienici.com/photo/immo-facile-50380739_media.immo-facile.com_office_sbi_catalog_images_pr_p_5_0_3_8_0_7_3_9_50380739a.jpg_DATEMAJ_19_07_2022-20_04_47', 'og:site_name': 'Bien’ici', 'og:description': 'Achetez cette maison de 9\xa0pièces et 235\xa0m² à Plérin', 'image': 'https://file.bienici.com/photo/immo-facile-50380739_media.immo-facile.com_office_sbi_catalog_images_pr_p_5_0_3_8_0_7_3_9_50380739a.jpg_DATEMAJ_19_07_2022-20_04_47', 'og:locale': 'fr_FR', 'article:tag': 'buy', 'twitter:card': 'summary_large_image', 'og:image': 'https://file.bienici.com/photo/immo-facile-50380739_media.immo-facile.com_office_sbi_catalog_images_pr_p_5_0_3_8_0_7_3_9_50380739a.jpg_DATEMAJ_19_07_2022-20_04_47', 'og:type': 'website', 'og:url': 'https://www.bienici.com/annonce/vente/plerin/maison/9pieces/immo-facile-50380739', 'apple-mobile-web-app-title': 'Bien’ici', 'viewport': 'width=device-width, initial-scale=1.0, user-scalable=no', 'twitter:description': 'Achetez cette maison de 9\xa0pièces et 235\xa0m² à Plérin'}]}, 'formattedUrl': 'https://www.bienici.com/annonce/vente/.../maison/.../immo-facile-50380739', 'title': 'Achat maison 9 pièces 235 m², Plérin - 760 900 €', 'displayLink': 'www.bienici.com', 'htmlTitle': 'Achat <b>maison</b> 9 pièces 235 m², <b>Plérin</b> - 760 900 €', 'htmlFormattedUrl': 'https://www.bienici.com/annonce/vente/.../<b>maison</b>/.../immo-facile-50380739'}}}]
num_to_return = 2
for i in range(num_to_return):

    result = search_results[id==i]
    details = fetch_appbuilder_details(result)



ERROR:root:An Error occurred while fetching appbuilder details: 'str' object is not callable
ERROR:root:An Error occurred while fetching appbuilder details: 'str' object is not callable


In [21]:
import os
from datetime import datetime

from langchain.llms import VertexAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1:  LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
llm = VertexAI( callbacks=callbacks)

In [24]:
from langchain.agents import initialize_agent, load_tools

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents.agent_types import AgentType
from langchain.llms import VertexAI

appbuilderTools = GoogleAppBuilderTool()

tools = appbuilderTools


agent = initialize_agent(
    [tools],
    llm=VertexAI(temperature=0, max_tokens=1000),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={"handle_parsing_errors": True},
    callbacks=callbacks,

)
agent.run(
   # "What is the more expensive restaurant in the birth place of Zinedine zidane ? "
   "maison à vendre plérin 2 voiture "
)

aim_callback.flush_tracker(langchain_asset=agent, reset=False, finish=True)




> Entering new  chain...
I need to find a house for sale in Plérin with 2 cars
Action: google_appbuilder
Action Input: maison à vendre plérin 2 voiture{'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_TYPE_UNSUPPORTED', 'metadata': {'service': 'discoveryengine.googleapis.com', 'method': 'google.cloud.discoveryengine.v1alpha.SearchService.Search'}}]}}


KeyError: 0

# gradio interface used by the demo

In [3]:
import gradio as gr

def process(query):
    return genAppBuilder(query)

import gradio as gr



with gr.Blocks(theme=gr.themes.Soft()) as demo:  
    with gr.Row():
        with gr.Column():
            query = gr.Textbox(label="query")
            greet_btn = gr.Button("search")
        with gr.Column():    
            title = gr.Textbox(label="Title")
            link = gr.Textbox(label="Link")
            snippet = gr.Textbox(label="Snippet")
            description = gr.Textbox(label="Description")
            image = gr.Textbox(label="Image")



    greet_btn.click(fn=process, inputs=query, outputs=[title, link, snippet, description, image])

demo.launch(debug=True, share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://8d1409dc40694f6a38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


{'results': [{'id': '0', 'document': {'name': 'projects/210552312048/locations/global/collections/default_collection/dataStores/interflora_website/branches/0/documents/0', 'id': '0', 'derivedStructData': {'htmlFormattedUrl': 'https://www.interflora.fr/', 'title': 'Interflora : livraison de fleurs à domicile en 4H', 'formattedUrl': 'https://www.interflora.fr/', 'snippets': [{'htmlSnippet': 'Découvrez nos bouquets de <b>fleurs</b> dès 22,95€. Livraison dans toute la France dès 6,95€, en 4H à Paris. Livraison 7J/7, dimanche et les jours fériés.', 'snippet': 'Découvrez nos bouquets de fleurs dès 22,95€. Livraison dans toute la France dès 6,95€, en 4H à Paris. Livraison 7J/7, dimanche et les jours fériés.'}], 'displayLink': 'www.interflora.fr', 'link': 'https://www.interflora.fr/', 'pagemap': {'metatags': [{'theme-color': '#303938', 'twitter:card': 'summary_large_image', 'twitter:creator': '@Interflora', 'og:type': 'website', 'twitter:image': 'https://blog.interflora.fr/wp-content/uploads/2

# example d'utilisation de l'api

In [ ]:
from gradio_client import Client

client_genappbuilder = Client("https://df5429029f7447730a.gradio.live")

client_genappbuilder.view_api()

client_genappbuilder.predict("fleur",  fn_index=0)

